# Agents

```{include} ../warning.md
```

AutoGen AgentChat 提供了一组预设代理，每个代理在响应消息的方式上都有所不同。
所有代理都共享以下属性和方法：

- {py:attr}`~autogen_agentchat.agents.BaseChatAgent.name`: 代理的唯一名称。
- {py:attr}`~autogen_agentchat.agents.BaseChatAgent.description`: 代理的文本描述。
- {py:meth}`~autogen_agentchat.agents.BaseChatAgent.on_messages`: 向代理发送一系列{py:class}`~autogen_agentchat.messages.ChatMessage`  并获得 {py:class}`~autogen_agentchat.base.Response`.
- {py:meth}`~autogen_agentchat.agents.BaseChatAgent.on_messages_stream`: 与{py:meth}`~autogen_agentchat.agents.BaseChatAgent.on_messages`  相同，但返回一个迭代器，包含{py:class}`~autogen_agentchat.messages.AgentMessage` ，最后一项是 {py:class}`~autogen_agentchat.base.Response` 。
- {py:meth}`~autogen_agentchat.agents.BaseChatAgent.on_reset`: 将代理重置为初始状态。

有关 AgentChat 消息类型的更多信息，请参见 {py:mod}`autogen_agentchat.messages`。


## Assistant Agent

{py:class}`~autogen_agentchat.agents.AssistantAgent` 是一个内置代理，
它使用具有使用工具能力的语言模型。

In [1]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_core.base import CancellationToken
from autogen_ext.models import OpenAIChatCompletionClient

# Define a tool that searches the web for information.
async def web_search(query: str) -> str:
    """Find information on the web"""
    return "AutoGen is a programming framework for building multi-agent applications."

# Create an agent that uses the OpenAI GPT-4o model.
model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    api_key="sk-",
)
agent = AssistantAgent(
    name="assistant",
    model_client=model_client,
    tools=[web_search],
    system_message="Use tools to solve tasks.",
)

我们可以调用 {py:meth}`~autogen_agentchat.agents.AssistantAgent.on_messages` 
方法来获取代理对消息的响应。

In [2]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="Find information on AutoGen", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()

[ToolCallMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=61, completion_tokens=15), content=[FunctionCall(id='call_90iZvYWtw8YLOvzPIhyYFCaw', arguments='{"query":"AutoGen"}', name='web_search')]), ToolCallResultMessage(source='assistant', models_usage=None, content=[FunctionExecutionResult(content='AutoGen is a programming framework for building multi-agent applications.', call_id='call_90iZvYWtw8YLOvzPIhyYFCaw')])]
source='assistant' models_usage=RequestUsage(prompt_tokens=92, completion_tokens=14) content='AutoGen is a programming framework used for building multi-agent applications.'


调用 {py:meth}`~autogen_agentchat.agents.AssistantAgent.on_messages` 方法
会返回一个 {py:class}`~autogen_agentchat.base.Response`，
其中包含代理的最终响应（在 {py:attr}`~autogen_agentchat.base.Response.chat_message` 属性中），
以及一个内部消息列表（在 {py:attr}`~autogen_agentchat.base.Response.inner_messages` 属性中），
这些内部消息存储了代理产生最终响应的"思考过程"。

### 流式消息

我们还可以使用 {py:meth}`~autogen_agentchat.agents.AssistantAgent.on_messages_stream` 方法
来实时流式获取代理生成的每条消息，
并使用 {py:class}`~autogen_agentchat.task.Console` 将消息
实时打印到控制台。

In [3]:
from autogen_agentchat.task import Console


async def assistant_run_stream() -> None:
    # Option 1: read each message from the stream.
    # async for message in agent.on_messages_stream(
    #     [TextMessage(content="Find information on AutoGen", source="user")],
    #     cancellation_token=CancellationToken(),
    # ):
    #     print(message)

    # Option 2: use Console to print all messages as they appear.
    await Console(
        agent.on_messages_stream(
            [TextMessage(content="Find information on AutoGen", source="user")],
            cancellation_token=CancellationToken(),
        )
    )


# Use asyncio.run(assistant_run_stream()) when running in a script.
await assistant_run_stream()

---------- assistant ----------
[FunctionCall(id='call_VfVDAK3OBqB9BQ6emRoraTKe', arguments='{"query":"AutoGen programming framework"}', name='web_search')]
[Prompt tokens: 127, Completion tokens: 17]
---------- assistant ----------
[FunctionExecutionResult(content='AutoGen is a programming framework for building multi-agent applications.', call_id='call_VfVDAK3OBqB9BQ6emRoraTKe')]
---------- assistant ----------
AutoGen is a programming framework designed for developing multi-agent applications.
[Prompt tokens: 160, Completion tokens: 14]
---------- Summary ----------
Number of inner messages: 2
Total prompt tokens: 287
Total completion tokens: 31
Duration: 1.88 seconds


{py:meth}`~autogen_agentchat.agents.AssistantAgent.on_messages_stream` 方法返回一个异步生成器，它会生成代理产生的每条内部消息，
最后一项是 {py:attr}`~autogen_agentchat.base.Response.chat_message` 属性中的最终响应消息。

从消息中可以看到，助手代理使用了 `web_search` 工具来搜索信息，并使用搜索结果进行响应。

### 理解工具调用

大型语言模型（LLMs）通常仅限于生成文本或代码响应。然而，许多复杂任务都能从使用外部工具执行特定操作中受益，比如从 API 或数据库获取数据。

为了解决这个限制，现代 LLMs 现在可以接受一个可用工具模式列表（工具及其参数的描述）并生成工具调用消息。这种能力被称为**Tool Calling**或**Function Calling**，正在成为构建智能代理应用程序的流行模式。

有关工具调用的更多信息，请参阅 [OpenAI](https://platform.openai.com/docs/guides/function-calling) 和 [Anthropic](https://docs.anthropic.com/en/docs/build-with-claude/tool-use) 的文档。

## 其他预设代理

以下预设代理可供使用：

- {py:class}`~autogen_agentchat.agents.CodeExecutorAgent`: 一个可以执行代码的代理。
- {py:class}`~autogen_ext.agents.MultimodalWebSurfer`: 一个多模态代理，可以搜索网页并访问网页获取信息。

## 下一步

现在我们已经讨论了如何使用 {py:class}`~autogen_agentchat.agents.AssistantAgent`，
我们可以继续下一节来学习如何使用 AgentChat 的团队功能。

<!-- ## CodingAssistantAgent

Generates responses (text and code) using an LLM upon receipt of a message. It takes a `system_message` argument that defines or sets the tone for how the agent's LLM should respond. 

```python

writing_assistant_agent = CodingAssistantAgent(
    name="writing_assistant_agent",
    system_message="You are a helpful assistant that solve tasks by generating text responses and code.",
    model_client=model_client,
)
`

We can explore or test the behavior of the agent by sending a message to it using the  {py:meth}`~autogen_agentchat.agents.BaseChatAgent.on_messages`  method. 

```python
result = await writing_assistant_agent.on_messages(
    messages=[
        TextMessage(content="What is the weather right now in France?", source="user"),
    ],
    cancellation_token=CancellationToken(),
)
print(result) -->